In [1]:
! pip install -q tensorflow-gpu==2.0.0.alpha0
import tensorflow as tf
print(tf.__version__)

    100% |████████████████████████████████| 332.1MB 51kB/s 
    100% |████████████████████████████████| 3.0MB 8.0MB/s 
    100% |████████████████████████████████| 419kB 11.6MB/s 
    100% |████████████████████████████████| 61kB 25.3MB/s 
2.0.0-alpha0


In [0]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt

In [0]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [10]:
print(x_train.shape)

(60000, 28, 28)


In [0]:
x_train = np.reshape(x_train, (len(x_train), 28,28,1))
x_test = np.reshape(x_test, (len(x_test), 28,28,1))

11493376/11490434 [==============================] - 0s 0us/step


In [24]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_train.shape

(60000, 784)

In [0]:
class Encoder(tf.keras.layers.Layer):
    '''Encodes a digit from the MNIST dataset'''
    
    def __init__(self,
                n_dims,
                name='encoder',
                **kwargs):
        super(Encoder,self).__init__(name=name, **kwargs)
        self.n_dims = n_dims
        self.n_layers = 1
        self.encode_layer = layers.Dense(n_dims, activation='relu')
        
    @tf.function        
    def call(self, inputs):
        return self.encode_layer(inputs)
        
class Decoder(tf.keras.layers.Layer):
    '''Decodes a digit from the MNIST dataset'''

    def __init__(self,
                n_dims,
                name='decoder',
                **kwargs):
        super(Decoder,self).__init__(name=name, **kwargs)
        self.n_dims = n_dims
        self.n_layers = len(n_dims)
        self.decode_middle = layers.Dense(n_dims[0], activation='relu')
        self.recon_layer = layers.Dense(n_dims[1], activation='sigmoid')
        
    @tf.function        
    def call(self, inputs):
        x = self.decode_middle(inputs)
        return self.recon_layer(x)
        
        
    
class Autoencoder(tf.keras.Model):
    '''Vanilla Autoencoder for MNIST digits'''
    
    def __init__(self,
                 n_dims=[200, 392, 784],
                 name='autoencoder',
                 **kwargs):
        super(Autoencoder, self).__init__(name=name, **kwargs)
        self.n_dims = n_dims
        self.encoder = Encoder(n_dims[0])
        self.decoder = Decoder([n_dims[1], n_dims[2]])
        
    @tf.function        
    def call(self, inputs):
        x = self.encoder(inputs)
        return self.decoder(x)
        

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
import tensorflow.keras.layers as layers

# Remove this comment from autoencoder_models.Autoencoder import Autoencoder

mnist = tf.keras.datasets.mnist


def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test


def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index:(start_index + batch_size)]


(X_train, _), (X_test, _) = mnist.load_data()
X_train = tf.cast(np.reshape(X_train, (X_train.shape[0], X_train.shape[1] * X_train.shape[2])), tf.float64)
X_test = tf.cast(np.reshape(X_test, (X_test.shape[0], X_test.shape[1] * X_test.shape[2])), tf.float64)

X_train, X_test = standard_scale(X_train, X_test)

train_data = tf.data.Dataset.from_tensor_slices(X_train).batch(128).shuffle(buffer_size=1024)
test_data = tf.data.Dataset.from_tensor_slices(X_test).batch(128).shuffle(buffer_size=512)

n_samples = int(len(X_train) + len(X_test))
training_epochs = 20
batch_size = 128
display_step = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
mse_loss = tf.keras.losses.MeanSquaredError()
loss_metric = tf.keras.metrics.Mean()

autoencoder = Autoencoder([200, 394, 784])

In [84]:
# Iterate over epochs.
for epoch in range(10):
    print(f'Epoch {epoch+1}')

  # Iterate over the batches of the dataset.
    for step, x_batch in enumerate(train_data):
        with tf.GradientTape() as tape:
          recon = autoencoder(x_batch)
          loss = mse_loss(x_batch, recon)

        grads = tape.gradient(loss, autoencoder.trainable_variables)
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_variables))

        loss_metric(loss)

        if step % 100 == 0:
          print(f'Step {step}: mean loss = {loss_metric.result()}')

Epoch 1
Step 0: mean loss = 1.4021320343017578
Step 100: mean loss = 0.8105310201644897
Step 200: mean loss = 0.7495337128639221
Step 300: mean loss = 0.7320978045463562
Step 400: mean loss = 0.7485438585281372
Epoch 2
Step 0: mean loss = 0.7548137903213501
Step 100: mean loss = 0.7507814764976501
Step 200: mean loss = 0.7402821779251099
Step 300: mean loss = 0.7350123524665833
Step 400: mean loss = 0.7428106069564819
Epoch 3
Step 0: mean loss = 0.7468051910400391
Step 100: mean loss = 0.7459006309509277
Step 200: mean loss = 0.7406624555587769
Step 300: mean loss = 0.7377558350563049
Step 400: mean loss = 0.742964506149292


IndentationError: ignored

In [67]:
ae = Autoencoder([200, 392, 784])
ae.compile(optimizer=tf.optimizers.Adam(0.1), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])
ae.fit(X_train, X_train, batch_size=64, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 4s 60us/sample - loss: 0.9148 - accuracy: 0.0000e+00
Epoch 2/5
60000/60000 [==============================] - 3s 58us/sample - loss: 0.9145 - accuracy: 0.0000e+00
Epoch 3/5
60000/60000 [==============================] - 3s 57us/sample - loss: 0.9145 - accuracy: 0.0000e+00
Epoch 4/5
60000/60000 [==============================] - 3s 57us/sample - loss: 0.9145 - accuracy: 0.0000e+00
Epoch 5/5
60000/60000 [==============================] - 3s 57us/sample - loss: 0.9145 - accuracy: 0.0000e+00


In [0]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)

        # Fit training using batch data
        cost = autoencoder.partial_fit(batch_xs)
        # Compute average loss
        avg_cost += cost / n_samples * batch_size

    # Display logs per epoch step
    if epoch % display_step == 0:
        print("Epoch:", '%d,' % (epoch + 1),
              "Cost:", "{:.9f}".format(avg_cost))

#print("Total cost: " + str(autoencoder.calc_total_cost(X_test)))

In [72]:
class Sampling(layers.Layer):
  """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
  """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

  def __init__(self,
               latent_dim=32,
               intermediate_dim=64,
               name='encoder',
               **kwargs):
    super(Encoder, self).__init__(name=name, **kwargs)
    self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
    self.dense_mean = layers.Dense(latent_dim)
    self.dense_log_var = layers.Dense(latent_dim)
    self.sampling = Sampling()

  def call(self, inputs):
    x = self.dense_proj(inputs)
    z_mean = self.dense_mean(x)
    z_log_var = self.dense_log_var(x)
    z = self.sampling((z_mean, z_log_var))
    return z_mean, z_log_var, z


class Decoder(layers.Layer):
  """Converts z, the encoded digit vector, back into a readable digit."""

  def __init__(self,
               original_dim,
               intermediate_dim=64,
               name='decoder',
               **kwargs):
    super(Decoder, self).__init__(name=name, **kwargs)
    self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
    self.dense_output = layers.Dense(original_dim, activation='sigmoid')

  def call(self, inputs):
    x = self.dense_proj(inputs)
    return self.dense_output(x)


class VariationalAutoEncoder(tf.keras.Model):
  """Combines the encoder and decoder into an end-to-end model for training."""

  def __init__(self,
               original_dim,
               intermediate_dim=64,
               latent_dim=32,
               name='autoencoder',
               **kwargs):
    super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
    self.original_dim = original_dim
    self.encoder = Encoder(latent_dim=latent_dim,
                           intermediate_dim=intermediate_dim)
    self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

  def call(self, inputs):
    z_mean, z_log_var, z = self.encoder(inputs)
    reconstructed = self.decoder(z)
    # Add KL divergence regularization loss.
    kl_loss = - 0.5 * tf.reduce_mean(
        z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
    self.add_loss(kl_loss)
    return reconstructed


original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255

vae = VariationalAutoEncoder(784, 64, 32)

# Iterate over epochs.
for epoch in range(3):
  print('Start of epoch %d' % (epoch,))

  # Iterate over the batches of the dataset.
  for step, x_batch_train in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      reconstructed = vae(x_batch_train)
      # Compute reconstruction loss
      loss = mse_loss_fn(x_batch_train, reconstructed)
      loss += sum(vae.losses)  # Add KLD regularization loss

    grads = tape.gradient(loss, vae.trainable_variables)
    optimizer.apply_gradients(zip(grads, vae.trainable_variables))

    loss_metric(loss)

    if step % 100 == 0:
      print('step %s: mean loss = %s' % (step, loss_metric.result()))
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
# vae.fit(x_train, x_train, epochs=20, batch_size=64)

Start of epoch 0
step 0: mean loss = tf.Tensor(0.34475994, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.12629451, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.09963466, shape=(), dtype=float32)
step 300: mean loss = tf.Tensor(0.08944599, shape=(), dtype=float32)
step 400: mean loss = tf.Tensor(0.08441953, shape=(), dtype=float32)
step 500: mean loss = tf.Tensor(0.081048794, shape=(), dtype=float32)
step 600: mean loss = tf.Tensor(0.07887767, shape=(), dtype=float32)
step 700: mean loss = tf.Tensor(0.07725732, shape=(), dtype=float32)
step 800: mean loss = tf.Tensor(0.076082826, shape=(), dtype=float32)
step 900: mean loss = tf.Tensor(0.07504082, shape=(), dtype=float32)
Start of epoch 1
step 0: mean loss = tf.Tensor(0.07474806, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.0740752, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.07357777, shape=(), dtype=float32)
step 300: mean loss = tf.Tensor(0.07309231, shape=(), dtype=float32)
ste